In [ ]:
import pandas as pd
import numpy as np

df = pd.read_csv("../input/tabular-playground-series-sep-2021/train.csv")


train1 = pd.read_csv("../input/tpssept/train/cat_train_standard_1.csv")
train2 = pd.read_csv("../input/tpssept/train/cat_train_standard_2.csv")
train3 = pd.read_csv("../input/tpssept/train/lgb_train_minmax_2.csv")
train4 = pd.read_csv("../input/tpssept/train/lgb_train_minmax.csv")
train5 = pd.read_csv("../input/tpssept/train/xgb_train_minmax.csv")
train6 = pd.read_csv("../input/tpssept/train/xgb_train_standard_2.csv")
train7 = pd.read_csv("../input/tpssept/train/xgb_train_standard.csv")


test1 = pd.read_csv("../input/tpssept/test/submission_cat_1.csv")
test2 = pd.read_csv("../input/tpssept/test/submission_cat_2.csv")
test3 = pd.read_csv("../input/tpssept/test/submission_lgb_2.csv")
test4 = pd.read_csv("../input/tpssept/test/submission_lgb_1.csv")
test5 = pd.read_csv("../input/tpssept/test/submission_xgb_3.csv")
test6 = pd.read_csv("../input/tpssept/test/submission_xgb_2.csv")
test7 = pd.read_csv("../input/tpssept/submission_xgb_1.csv")

train = pd.DataFrame()
train['id'] = df['id']
train['model1'] = train1['lgb_1']
train['model2'] = train2['lgb_1']
train['model3'] = train3['lgb_1']
train['model4'] = train4['lgb_1']
train['model5'] = train5['lgb_1']
train['model6'] = train6['lgb_1']
train['model7'] = train7['lgb_1']
train['claim'] = df['claim']

test = pd.DataFrame()
test['id'] = test1['id']
test['model1'] = test1['claim']
test['model2'] = test2['claim']
test['model3'] = test3['claim']
test['model4'] = test4['claim']
test['model5'] = test5['claim']
test['model6'] = test6['claim']
test['model7'] = test7['claim']
useful_features = [x for x in test.columns if x not in ['id','claim']]

train.head()

In [ ]:
test.head()

In [ ]:
train_level2 = pd.DataFrame()
test_level2 = pd.DataFrame()

train_level2['id'] = train['id']
test_level2['id'] = test['id']

In [ ]:
X = train[useful_features].values
y = train['claim'].values

from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import StratifiedKFold 
from sklearn.metrics import roc_auc_score
fold = StratifiedKFold(n_splits=10,shuffle=True,random_state=42)

i = 1
oof = np.zeros(df.shape[0])

lgb_preds = []
test_score = []
train_score = []


for train_id,test_id in fold.split(train[useful_features],train['claim']):
        
        
        
        #training and testing data
        xtrain ,xtest = X[train_id],X[test_id]
        ytrain,ytest = y[train_id],y[test_id]
        x_test_data = test[useful_features].values
        
        #model instance 
        model = LogisticRegression(solver='saga',
            max_iter=1500,
            random_state=42)
        
        #fitting the model
        model.fit(xtrain, ytrain)
        
        #predicting train and test data
        test_pred = model.predict_proba(xtest)[:, 1]
        train_pred = model.predict_proba(xtrain)[:,1] 
        oof[test_id] = model.predict_proba(xtest)[:,1]
        
        test_auc = roc_auc_score(ytest, test_pred)
        train_auc = roc_auc_score(ytrain,train_pred)
        
        #appending into the list
        train_score.append(train_auc)
        test_score.append(test_auc)
        
        #printing the scores
        print("---------- Fold- {} ---------".format(i))
        print("Train roc_auc_score is {}".format(train_auc))
        print("Test roc_auc_score is {}".format(test_auc))
        i = i+1
        
        lgb_preds.append(model.predict_proba(x_test_data)[:,-1])
          
print("----------All folds Done-----------")
print("Average train auc : {} with standard devation : {}".format(np.mean(train_score),np.std(train_score)))
print("Average test auc : {} with standard devation : {}".format(np.mean(test_score),np.std(test_score)))
lgb_preds = np.mean(np.column_stack(lgb_preds), axis=1)

sample = pd.read_csv('../input/tabular-playground-series-sep-2021/sample_solution.csv')
sample['claim'] = lgb_preds
sample.to_csv('submission.csv',index = False)


train_level2['oof_1'] = oof
test_level2['oof_1'] = lgb_preds


In [ ]:
X = train[useful_features].values
y = train['claim'].values

xgb_params = {'objective': 'binary:logistic',
              'use_label_encoder': False,
                'n_estimators': 13000, 
              'learning_rate': 0.01,  
              'max_depth': 6,
             'booster': 'gbtree',
             'random_state': 42}

from xgboost import XGBClassifier
from sklearn.model_selection import StratifiedKFold 
from sklearn.metrics import roc_auc_score
fold = StratifiedKFold(n_splits=10,shuffle=True,random_state=42)

i = 1
oof = np.zeros(df.shape[0])

lgb_preds = []
test_score = []
train_score = []


for train_id,test_id in fold.split(train[useful_features],train['claim']):
        
        
        
        #training and testing data
        xtrain ,xtest = X[train_id],X[test_id]
        ytrain,ytest = y[train_id],y[test_id]
        x_test_data = test[useful_features].values
        
        #model instance 
        model = XGBClassifier(**xgb_params,
                              tree_method='gpu_hist', 
                predictor='gpu_predictor')
        
        #fitting the model
        model.fit(xtrain, ytrain, 
                  verbose=500,
                 eval_set=[(xtest,ytest)],
                  eval_metric=['logloss','auc'],
                 early_stopping_rounds=500)
        
        #predicting train and test data
        test_pred = model.predict_proba(xtest)[:, 1]
        train_pred = model.predict_proba(xtrain)[:,1] 
        oof[test_id] = model.predict_proba(xtest)[:,1]
        
        test_auc = roc_auc_score(ytest, test_pred)
        train_auc = roc_auc_score(ytrain,train_pred)
        
        #appending into the list
        train_score.append(train_auc)
        test_score.append(test_auc)
        
        #printing the scores
        print("---------- Fold- {} ---------".format(i))
        print("Train roc_auc_score is {}".format(train_auc))
        print("Test roc_auc_score is {}".format(test_auc))
        i = i+1
        
        lgb_preds.append(model.predict_proba(x_test_data)[:,-1])
          
print("----------All folds Done-----------")
print("Average train auc : {} with standard devation : {}".format(np.mean(train_score),np.std(train_score)))
print("Average test auc : {} with standard devation : {}".format(np.mean(test_score),np.std(test_score)))
lgb_preds = np.mean(np.column_stack(lgb_preds), axis=1)

sample = pd.read_csv('../input/tabular-playground-series-sep-2021/sample_solution.csv')
sample['claim'] = lgb_preds
sample.to_csv('submission_2.csv',index = False)


train_level2['oof_2'] = oof
test_level2['oof_2'] = lgb_preds


In [ ]:
X = train[useful_features].values
y = train['claim'].values

catb_params = {
    'eval_metric' : 'AUC',
    'depth' : 5,
    'learning_rate' : 0.1,
    'iterations' : 12000,
    'loss_function' : 'CrossEntropy',
    'task_type' : 'GPU',
    'devices' : '0',
    'verbose' : 0
}

from catboost import CatBoostClassifier
from sklearn.model_selection import StratifiedKFold 
from sklearn.metrics import roc_auc_score
fold = StratifiedKFold(n_splits=10,shuffle=True,random_state=42)

i = 1
oof = np.zeros(df.shape[0])

lgb_preds = []
test_score = []
train_score = []


for train_id,test_id in fold.split(train[useful_features],train['claim']):
        
        
        
        #training and testing data
        xtrain ,xtest = X[train_id],X[test_id]
        ytrain,ytest = y[train_id],y[test_id]
        x_test_data = test[useful_features].values
        
        #model instance 
        model =  CatBoostClassifier(**catb_params)
        
        #fitting the model
        model.fit(xtrain, ytrain, 
                 eval_set=[(xtest,ytest)],
                 early_stopping_rounds=500)
        
        #predicting train and test data
        test_pred = model.predict_proba(xtest)[:, 1]
        train_pred = model.predict_proba(xtrain)[:,1] 
        oof[test_id] = model.predict_proba(xtest)[:,1]
        
        test_auc = roc_auc_score(ytest, test_pred)
        train_auc = roc_auc_score(ytrain,train_pred)
        
        #appending into the list
        train_score.append(train_auc)
        test_score.append(test_auc)
        
        #printing the scores
        print("---------- Fold- {} ---------".format(i))
        print("Train roc_auc_score is {}".format(train_auc))
        print("Test roc_auc_score is {}".format(test_auc))
        i = i+1
        
        lgb_preds.append(model.predict_proba(x_test_data)[:,-1])
          
print("----------All folds Done-----------")
print("Average train auc : {} with standard devation : {}".format(np.mean(train_score),np.std(train_score)))
print("Average test auc : {} with standard devation : {}".format(np.mean(test_score),np.std(test_score)))
lgb_preds = np.mean(np.column_stack(lgb_preds), axis=1)

sample = pd.read_csv('../input/tabular-playground-series-sep-2021/sample_solution.csv')
sample['claim'] = lgb_preds
sample.to_csv('submission_3.csv',index = False)


train_level2['oof_3'] = oof
test_level2['oof_3'] = lgb_preds

In [ ]:
train_level2['claim'] = train['claim']
train_level2.to_csv('train_level_2.csv',index = False)
train_level2.head()

In [ ]:
test_level2.to_csv('test_level_2.csv',index = False)
test_level2.head()

In [ ]:
useful_features = [x for x in test_level2.columns if x not in ['id','claim']]
X = train_level2[useful_features].values
y = train_level2['claim'].values

xgb_params = {'objective': 'binary:logistic',
              'use_label_encoder': False,
                'n_estimators': 13000, 
              'learning_rate': 0.01,  
              'max_depth': 6,
             'booster': 'gbtree',
             'random_state': 42}

from xgboost import XGBClassifier
from sklearn.model_selection import StratifiedKFold 
from sklearn.metrics import roc_auc_score
fold = StratifiedKFold(n_splits=10,shuffle=True,random_state=42)

i = 1
oof = np.zeros(df.shape[0])

lgb_preds = []
test_score = []
train_score = []


for train_id,test_id in fold.split(train_level2[useful_features],train_level2['claim']):
        
        
        
        #training and testing data
        xtrain ,xtest = X[train_id],X[test_id]
        ytrain,ytest = y[train_id],y[test_id]
        x_test_data = test_level2[useful_features].values
        
        #model instance 
        model = XGBClassifier(**xgb_params,
                              tree_method='gpu_hist', 
                predictor='gpu_predictor')
        
        #fitting the model
        model.fit(xtrain, ytrain, 
                  verbose=500,
                 eval_set=[(xtest,ytest)],
                  eval_metric=['logloss','auc'],
                 early_stopping_rounds=500)
        
        #predicting train and test data
        test_pred = model.predict_proba(xtest)[:, 1]
        train_pred = model.predict_proba(xtrain)[:,1] 
        oof[test_id] = model.predict_proba(xtest)[:,1]
        
        test_auc = roc_auc_score(ytest, test_pred)
        train_auc = roc_auc_score(ytrain,train_pred)
        
        #appending into the list
        train_score.append(train_auc)
        test_score.append(test_auc)
        
        #printing the scores
        print("---------- Fold- {} ---------".format(i))
        print("Train roc_auc_score is {}".format(train_auc))
        print("Test roc_auc_score is {}".format(test_auc))
        i = i+1
        
        lgb_preds.append(model.predict_proba(x_test_data)[:,-1])
          
print("----------All folds Done-----------")
print("Average train auc : {} with standard devation : {}".format(np.mean(train_score),np.std(train_score)))
print("Average test auc : {} with standard devation : {}".format(np.mean(test_score),np.std(test_score)))
lgb_preds = np.mean(np.column_stack(lgb_preds), axis=1)

sample = pd.read_csv('../input/tabular-playground-series-sep-2021/sample_solution.csv')
sample['claim'] = lgb_preds
sample.to_csv('submission_4.csv',index = False)



In [ ]:
useful_features = [x for x in test_level2.columns if x not in ['id','claim']]
X = train_level2[useful_features].values
y = train_level2['claim'].values


from xgboost import XGBClassifier
from sklearn.model_selection import StratifiedKFold 
from sklearn.metrics import roc_auc_score
fold = StratifiedKFold(n_splits=10,shuffle=True,random_state=42)

i = 1
oof = np.zeros(df.shape[0])

lgb_preds = []
test_score = []
train_score = []


for train_id,test_id in fold.split(train_level2[useful_features],train_level2['claim']):
        
        
        
        #training and testing data
        xtrain ,xtest = X[train_id],X[test_id]
        ytrain,ytest = y[train_id],y[test_id]
        x_test_data = test_level2[useful_features].values
        
        #model instance 
        model = LogisticRegression(solver='saga',
            max_iter=1500,
            random_state=42)
        
        #fitting the model
        model.fit(xtrain, ytrain)
        
        #predicting train and test data
        test_pred = model.predict_proba(xtest)[:, 1]
        train_pred = model.predict_proba(xtrain)[:,1] 
        oof[test_id] = model.predict_proba(xtest)[:,1]
        
        test_auc = roc_auc_score(ytest, test_pred)
        train_auc = roc_auc_score(ytrain,train_pred)
        
        #appending into the list
        train_score.append(train_auc)
        test_score.append(test_auc)
        
        #printing the scores
        print("---------- Fold- {} ---------".format(i))
        print("Train roc_auc_score is {}".format(train_auc))
        print("Test roc_auc_score is {}".format(test_auc))
        i = i+1
        
        lgb_preds.append(model.predict_proba(x_test_data)[:,-1])
          
print("----------All folds Done-----------")
print("Average train auc : {} with standard devation : {}".format(np.mean(train_score),np.std(train_score)))
print("Average test auc : {} with standard devation : {}".format(np.mean(test_score),np.std(test_score)))
lgb_preds = np.mean(np.column_stack(lgb_preds), axis=1)

sample = pd.read_csv('../input/tabular-playground-series-sep-2021/sample_solution.csv')
sample['claim'] = lgb_preds
sample.to_csv('submission_5.csv',index = False)